In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#[64,128,256,512]
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.block1 = nn.Sequential(
            #in channel is 6 since we are concat the two types
            nn.Conv2d(in_channels=6,out_channels=64, kernel_size=4,stride=2,padding = 1, padding_mode='reflect'),
            nn.LeakyReLU(0.2),
        )
        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels=64,out_channels=128,kernel_size=4,stride=2,padding=1,padding_mode='reflect'),
            nn.BatchNorm2d(num_features = 128),
            nn.LeakyReLU(0.2)
        )
        self.block3 = nn.Sequential(
            nn.Conv2d(in_channels=128,out_channels=256,kernel_size=4,stride=2,padding=1,padding_mode='reflect'),
            nn.BatchNorm2d(num_features = 256),
            nn.LeakyReLU(0.2)
        )
        self.block4 = nn.Sequential(
            nn.Conv2d(in_channels=256,out_channels=512,kernel_size=4,stride=1,padding=1,padding_mode='reflect'),
            nn.BatchNorm2d(num_features = 512),
            nn.LeakyReLU(0.2)
        )
        self.block5 = nn.Conv2d(in_channels=512, out_channels=1, kernel_size=4,stride=1,padding=1,padding_mode='reflect')
    #x is input , y is our out put (fake or real)
    def forward(self,x,y):
        x = torch.cat([x,y], dim =1) # dim zero is ||, dim one is channels, we are concat channels
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        return x


In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.down1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=4, stride=2,padding = 1, padding_mode='reflect'),
            nn.LeakyReLU(0.2),
        )
        self.down2 = nn.Sequential(
            nn.Conv2d(in_channels=64 ,out_channels=128, kernel_size=4, stride=2,padding = 1, padding_mode='reflect'),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
                      
        )
        self.down3 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=4, stride=2,padding = 1, padding_mode='reflect'),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),
                      
        )
        self.down4 = nn.Sequential(
            nn.Conv2d(in_channels=256 ,out_channels=512, kernel_size=4, stride=2,padding = 1, padding_mode='reflect'),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2),
                      
        )
        self.down5 = nn.Sequential(
            nn.Conv2d(in_channels=512 ,out_channels=512, kernel_size=4, stride=2,padding = 1, padding_mode='reflect'),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2),
                      
        )
        self.down6 = nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=4, stride=2,padding = 1, padding_mode='reflect'),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2),
                      
        )
        self.down7 = nn.Sequential(
            nn.Conv2d(in_channels=512 ,out_channels=512, kernel_size=4, stride=2,padding = 1, padding_mode='reflect'),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2),
                      
        )

        self.bottleNeck = nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=4, stride=2,padding = 1, padding_mode='reflect'),
            nn.ReLU()
        )
        self.dropOut = nn.Dropout(0.5)

        self.up1 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=512, out_channels=512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU()
            #dropout
            
        )
        self.up2 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=1024, out_channels=512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU()
            #dropout
            
        )
        self.up3 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=1024, out_channels=512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU()
            #dropout
            
        )
        self.up4 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=1024, out_channels=512, kernel_size=4, stride=2,padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU()
          
            
        )
        self.up5 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=1024, out_channels=256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU()
          
            
        )
        self.up6 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=512, out_channels=128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU()
          
            
        )
        self.up7 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=256, out_channels=64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU()
          
            
        )
        self.out = nn.Sequential(
            nn.ConvTranspose2d(in_channels=128, out_channels=3, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
            
        )

    def forward(self,x):
        d1 = self.down1(x)
        d2 = self.down2(d1)
        d3 = self.down3(d2)
        d4 = self.down4(d3)
        d5 = self.down5(d4)
        d6 = self.down6(d5)
        d7 = self.down7(d6)
        bottleneck = self.bottleNeck(d7)

        u1 = self.up1(bottleneck)
        u1 =self.dropOut(u1)
        u2 = self.up2(torch.cat([u1,d7],1))
        u2 =self.dropOut(u2)
        u3 = self.up3(torch.cat([u2,d6],1))
        u3 = self.dropOut(u3)
        u4 = self.up4(torch.cat([u3,d5],1))
        u5 = self.up5(torch.cat([u4,d4],1))
        u6 = self.up6(torch.cat([u5,d3],1))
        u7 = self.up7(torch.cat([u6,d2],1))
        output = self.out(torch.cat([u7,d1],1))
        return output

In [ ]:
import torch
from torchvision.transforms import v2
from PIL import Image
import os
import numpy as np
import torchvision.transforms as transforms
import torchvision.transforms.functional as F

transform = transforms.Compose([
    transforms.Resize(size=(256, 256)),
    #transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

class mapDataSet():
    def __init__(self, directory):
        self.directory = directory
        self.listImages = os.listdir(self.directory)
        #self.transform = transforms 

    def __len__(self):
        return len(self.listImages)

    def __getitem__(self, index):
      
        imageFile = self.listImages[index]
        imagePath = os.path.join(self.directory, imageFile)
        image = Image.open(imagePath)  
        width, height = image.size
        
        inputImage = image.crop((0, 0, width // 2, height))  
        targetImage = image.crop((width // 2, 0, width, height))  

        if torch.rand(1) < 0.5:  
            inputImage = F.hflip(inputImage)
            targetImage = F.hflip(targetImage)
        
        inputImage = transform(inputImage)
        targetImage = transform(targetImage)

        return inputImage, targetImage

        
    

In [ ]:

def trainingFunction(generator, discriminator, trainingLoader, optimisorDiscriminator, optimisorGenerator, l1Loss, binaryCrossEntropy, l1Lambda,epoch):
    loop = tqdm(trainingLoader, leave = True)
    if (epoch + 1) % 10 == 0:
        saveDirectory = '/kaggle/working/'
        os.makedirs(saveDirectory, exist_ok=True)
        
        generator_path = os.path.join(saveDirectory, f'generator{epoch}.pth')
        discriminator_path = os.path.join(saveDirectory, f'discriminator{epoch}.pth')
        
        torch.save(generator.state_dict(), generator_path)
        torch.save(discriminator.state_dict(), discriminator_path)
        
        print(f'Models saved at epoch {epoch + 1}')

    for index, (x,y) in enumerate(loop):
        x = x.to(device)
        y = y.to(device)

        yGenerated = generator(x)
        dReal = discriminator(x,y)
        dFake = discriminator(x, yGenerated.detach()) #loss.backward(retain_grapg = True)
        discriminatorRealLoss  = binaryCrossEntropy(dReal, torch.ones_like(dReal))
        discriminatorFakeLoss  = binaryCrossEntropy(dFake, torch.ones_like(dFake))
        discriminatorLoss = (discriminatorFakeLoss + discriminatorRealLoss) / 2 # paper, experiment with no devision

    discriminator.zero_grad()
    discriminatorLoss.backward()
    optimisorDiscriminator.step()


    dFake = discriminator(x, yGenerated.detach())
    generatorFakeLoss = binaryCrossEntropy(dFake, torch.ones_like(dFake))
    l1 = l1Loss(yGenerated, y) + l1Lambda
    generatorLoss = generatorFakeLoss + l1

    generator.zero_grad()
    generatorLoss.backward()
    optimisorGenerator.step()
        

In [ ]:

generator = Generator().to(device)
discriminator =  Discriminator().to(device)
optimisorDiscriminator = optim.Adam(discriminator.parameters(), lr = 2e-4, betas = (0.5,0.999))
optimisorGenerator = optim.Adam(generator.parameters() , lr = 2e-4, betas = (0.5, 0.999))
binaryCrossEntropy = nn.BCEWithLogitsLoss()
l1Loss = nn.L1Loss()
l1Lambda = 100

trainingDataSet = mapDataSet('/kaggle/input/mapdataset/maps/train')
testDataSet = mapDataSet('/kaggle/input/mapdataset/maps/val')
trainingLoader = DataLoader(trainingDataSet, batch_size = 4, shuffle = True, num_workers = 1)
testLoader = DataLoader(testDataSet, batch_size = 1, shuffle = False, num_workers = 1)
epochs = 15
for i in range(epochs):

    trainingFunction(generator, discriminator, trainingLoader, optimisorDiscriminator, optimisorGenerator, l1Loss, binaryCrossEntropy, l1Lambda,i+101)

In [ ]:
'''import torch
import os

def loadModelOnly(generator, discriminator, saveDirectory):
    generatorPath = os.path.join(saveDirectory, 'generator.pth')
    discriminatorPath = os.path.join(saveDirectory, 'discriminator.pth')
    
    if os.path.exists(generatorPath) and os.path.exists(discriminatorPath):
        generator.load_state_dict(torch.load(generatorPath))
        discriminator.load_state_dict(torch.load(discriminatorPath))
    

def trainingFunctionCont(generator, discriminator, trainingLoader, optimisorDiscriminator, optimisorGenerator, l1Loss, binaryCrossEntropy, l1Lambda, epoch, saveDirectory='/kaggle/working/'):
    loop = tqdm(trainingLoader, leave=False)
    
    if (epoch + 1) % 10 == 0:
        os.makedirs(saveDirectory, exist_ok=True)
        
        generator_path = os.path.join(saveDirectory, 'generator.pth')
        discriminator_path = os.path.join(saveDirectory, 'discriminator.pth')
        
        torch.save(generator.state_dict(), generator_path)
        torch.save(discriminator.state_dict(), discriminator_path)
        
        print(f'Models saved at epoch {epoch + 1}')

    
    for index, (x, y) in enumerate(loop):
        x = x.to(device)
        y = y.to(device)

        yGenerated = generator(x)
        
        dReal = discriminator(x, y)
        dFake = discriminator(x, yGenerated.detach())  # Detach to avoid updating the generator
        discriminatorRealLoss  = binaryCrossEntropy(dReal, torch.ones_like(dReal))
        discriminatorFakeLoss  = binaryCrossEntropy(dFake, torch.zeros_like(dFake))
        discriminatorLoss = (discriminatorFakeLoss + discriminatorRealLoss) / 2

        discriminator.zero_grad()
        discriminatorLoss.backward()
        optimisorDiscriminator.step()

        dFake = discriminator(x, yGenerated)
        generatorFakeLoss = binaryCrossEntropy(dFake, torch.ones_like(dFake))
        l1 = l1Loss(yGenerated, y) * l1Lambda
        generatorLoss = generatorFakeLoss + l1

        generator.zero_grad()
        generatorLoss.backward()
        optimisorGenerator.step()

saveDirectory = '/kaggle/working/'
loadModelOnly(generator, discriminator, saveDirectory)
epochs = 0
for i in range(epochs):
    trainingFunctionCont(generator, discriminator, trainingLoader, optimisorDiscriminator, optimisorGenerator, l1Loss, binaryCrossEntropy, l1Lambda, i, saveDirectory)
'''

In [ ]:
import matplotlib.pyplot as plt
import torchvision.transforms.functional as F
import torch
import torchmetrics
from sklearn.metrics import jaccard_score



def testFunction(generator, testLoader, l1Loss, maxImages=5):
    generator.eval() 
    iouValues = []
    maxImages = 0
    maxNeed 5
    with torch.no_grad():
        for index, (mapImage, aerialImage) in enumerate(testLoader):
            mapImage = mapImage.to(device)
            aerialImage = aerialImage.to(device)

            generatedAerial = generator(mapImage)

           
            iouValue = calculateIou(generatedAerial, aerialImage)
            iouValues.append(iouValue)
            
            showImages(mapImage, aerialImage, generatedAerial)
            maxImages+=1
            if(maxImages == maxNeed):
                break
            

  
    avgIou = sum(iouValues) / len(iouValues)
    print(f"Average IoU: {avgIou}")
    generator.train() 
    

def showImages(mapImage, aerialImage, generatedAerial):
    fig, axes = plt.subplots(1, 3, figsize=(12, 4))

    mapImage = mapImage.cpu().squeeze(0).permute(1, 2, 0) * 0.5 + 0.5  # unnormalize
    aerialImage = aerialImage.cpu().squeeze(0).permute(1, 2, 0) * 0.5 + 0.5  # unnormalize
    generatedAerial = generatedAerial.cpu().squeeze(0).permute(1, 2, 0) * 0.5 + 0.5  # unnormalize
    
    axes[0].imshow(mapImage)
    axes[0].set_title('Original Aerial')
    axes[0].axis('off')

    # Display the original aerial image
    axes[1].imshow(aerialImage)
    axes[1].set_title('Original Map')
    axes[1].axis('off')

    # Display the generated aerial image
    axes[2].imshow(generatedAerial)
    axes[2].set_title('Generated Map')
    axes[2].axis('off')

    plt.show()

def calculateIou(generated, real):
    threshold = 0.5
    generatedMask = (generated > threshold).int().cpu().numpy().flatten()
    realMask = (real > threshold).int().cpu().numpy().flatten()

    iou = jaccard_score(realMask, generatedMask, average='binary')
    return iou

testFunction(generator, testLoader, l1Loss, maxImages=5)


In [ ]:
epochs = 5
for i in range(epochs):

    trainingFunction(generator, discriminator, trainingLoader, optimisorDiscriminator, optimisorGenerator, l1Loss, binaryCrossEntropy, l1Lambda,i+101)

In [ ]:
testFunction(generator, testLoader, l1Loss, maxImages=5)


In [ ]:
epochs = 300
for i in range(epochs):

    trainingFunction(generator, discriminator, trainingLoader, optimisorDiscriminator, optimisorGenerator, l1Loss, binaryCrossEntropy, l1Lambda,i+101)

In [ ]:
testFunction(generator, testLoader, l1Loss, maxImages=5)


In [ ]:
!pip install torchvision


In [ ]:
import torch
import torch.nn.functional as F
from torchvision import transforms
import math
from torchvision.metrics import structural_similarity_index_measure as ssim

def calculate_psnr(generated, target):
    mse = F.mse_loss(generated, target)
    if mse == 0:
        return float('inf')
    return 20 * math.log10(1.0 / math.sqrt(mse))  # Assuming images are normalized to [0, 1]

def testFunction(generator, testLoader, l1Loss, maxImages=5):
    generator.eval()
    iouValues = []
    psnrValues = []
    ssimValues = []
    maxImages = 0

    with torch.no_grad():
        for index, (mapImage, aerialImage) in enumerate(testLoader):
            mapImage = mapImage.to(device)
            aerialImage = aerialImage.to(device)

            generatedAerial = generator(mapImage)

            # Calculate IoU
            iouValue = calculateIou(generatedAerial, aerialImage)
            iouValues.append(iouValue)

            # Calculate PSNR
            psnrValue = calculate_psnr(generatedAerial, aerialImage)
            psnrValues.append(psnrValue)

            # Calculate SSIM
            ssimValue = ssim(generatedAerial, aerialImage)
            ssimValues.append(ssimValue)

            # Show the images
            if maxImages < 5:
                showImages(mapImage, aerialImage, generatedAerial)
                maxImages += 1

    # Compute the average metrics
    avgIou = sum(iouValues) / len(iouValues)
    avgPsnr = sum(psnrValues) / len(psnrValues)
    avgSsim = sum(ssimValues) / len(ssimValues)

    # Print results
    print(f"Average IoU: {avgIou}")
    print(f"Average PSNR: {avgPsnr}")
    print(f"Average SSIM: {avgSsim}")

    generator.train()
maxImages = 5
(generator, testLoader, l1Loss,maxImages)